In [6]:
import numpy as np
import  os 
import cv2
import matplotlib.pyplot as plt
import imutils
import time
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

from imutils.video import VideoStream
from mtcnn import MTCNN

In [15]:
''' Initiate the face and the mask detectors '''

face_detector = MTCNN()
mask_detector = load_model('saved_model/mask_detection_model_160.h5')

In [18]:
def detect_mask_on_video(frame, face_detector, mask_detector):
    '''
        :param: frame -> frames that flow from video capture 
        :param: face_detector -> MTCNN face detector 
        :param: mask_detector -> the mask detection model we trained 
    '''
    
    detection_results = face_detector.detect_faces(frame)
    
    # Store the found faces, their coordinates and mask prediction values
    # in order to pass them to the video stream,
    # So we can draw the bounding boxes and show the prediction
    
    faces = []
    bounding_boxes = []
    mask_predictions = []
    
    for i in range(len(detection_results)):
        # Get the confidence of the face prediction
        confidence  = detection_results[i]['confidence']
        
        # Check if the model is more than 50% confident about finding a face
        if confidence > 0.5:
            
            # Get the coordinates of the bounding box(location of the face)
            x,y,w,h = detection_results[i]['box']
            
            
            # Append the bounding box locations to its storage
            bounding_boxes.append((x,y,w,h)) 
            
            # Slice the face from the image and preprocess it
            # Resize the face match the model`s shape
            # then pass it to the mask detection model 
            face = frame[y:y+h, x:x+w]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (160,160))
#             face = cv2.resize(face, (224,224))
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            
            # Store the face
            faces.append(face)
            
    # Check for a detected face on the frame        
    if len(faces) > 0:
        # put the extracted face to mask detection model 
        # to check for a mask
        mask_predictions = mask_detector.predict(faces)
            
    return (bounding_boxes, mask_predictions)

In [23]:
''' cv2.VideoCapture was too laggy, imutils.VideoStream performs slightly better  '''

vs = VideoStream(src=0).start()
time.sleep(2.0)

while(True):
    # Number of pixels affects the face detection and as a chain reaction, to the mask detection drastically
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    # Feed the frames to the detect_mask_on_video function simultaneously 
    (bounding_boxes, mask_predictions) = detect_mask_on_video(frame, face_detector, mask_detector)
    
    # Get the bbox location for each face, and assign the label and color 
    # according to the prediction
    for (b_box, m_pred) in zip(bounding_boxes, mask_predictions):
        (x,y,w,h)  = b_box
        (no_mask, mask) = m_pred
        
        label = ''
        color = (0,0,0)
        if mask > no_mask:
            label = 'MASK'
            color = (0,255,0)
        else:
            label = 'NO MASK'
            color = (0,0,255)
        
        # Put the label and the accuracy on top of the face
        cv2.putText(frame, label, (x, y-7),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x,y),(x+w, y+h), color, 2)
        
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cv2.destroyAllWindows()
vs.stream.release()
vs.stop()

In [63]:
## If video capture somehow gets stuck, run this to end it 
cv2.destroyAllWindows()
vs.stream.release()
vs.stop()